In [1]:
import os

import json
import joblib
import glob
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
print(f'tensorflow version : {tf.version.VERSION}')
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import *

tensorflow version : 2.9.1


In [2]:
# script should be in the same directory as the notebook
!pip install utils
import utils


In [3]:
def mean_dupes_drop_singles_filter_by_std( df, std_thres=0.03, filt_zeros=True):
    """ Within the SPC samples,  duplicates should be averaged, not dropped.
    This function:

    - averages the duplicates
    - calculates std over column
    -  filters the samples out with high std (high std could mean collection error)
    -  filters the samples out with 0 std (not a true duplicate)

    Parameters
    ----------
    :param: ``df`` : ``pd.DataFrame``
        Dataframe of SPC data.
    :param: ``std_thres`` : ``float``
        standard deviation threshold for acceptable data. Default = 0.01.
    :param: ``filt_zeros`` : ``bool``
        If True, filter out sample duplicates with a standard deviation of 0 --> unrealistic!

    Returns
    -------
    :return: ```pd.DataFrame``
        Dataframe in better shape.

    """
    # copy dataframe for std calculation
    df_std = df.copy(deep=True)
#     cls.__log.info("Shape of initial dataframe: %s", np.shape(df_std))

    # average duplicates
    df = df.groupby(df.index).mean()

    # calculate average std over column, maintain index
    df_std = df_std.groupby(df_std.index).std()
    # failed_indices = [(2138, 61433) ]
    # print(df_std.loc[failed_indices])
    # df_std =df_std.loc[failed_indices]
    # print(df.std(axis=0).sort_values(ascending =False).head(30))
    # calculate avg std series
    df_std = df_std.mean(axis=1)

    # create dataframe from avg std series
    df_std = pd.DataFrame(
        index=df_std.index.values, columns=["std"], data=df_std.values
    )
    print(df_std.sort_values(by='std',ascending =False).head())
    # print(df_std)
    df_std = df_std.dropna()
    # failed_indices = [(2138, 61433) ]  
    # df_std.loc[failed_indices]

#     cls.__log.info("Shape of dataframe w/o NaNs: %s", np.shape(df_std))

    # filter df by std, removing below thresh
    df_std = df_std[df_std["std"] < std_thres]
#     cls.__log.info("Shape of thresholded dataframe: %s", np.shape(df_std))

    if filt_zeros:
        df_std = df_std[df_std["std"] > 0]
#         cls.__log.info(
#             "Shape of thresholded dataframe with filtered zeros: %s",
#             np.shape(df_std),
#         )

    # merge the dataframes by index
    df_mrgd = pd.merge(df, df_std, left_index=True, right_index=True, how="outer")
#     cls.__log.info("Shape of merged dataframe by index: %s", np.shape(df_mrgd))

    # drop na's by row
    df_mrgd = df_mrgd.dropna()
#     cls.__log.info("Shape of merged dataframe w/o NaNs: %s", np.shape(df_mrgd))

    # drop std column
    df_mrgd = df_mrgd.drop(["std"], axis=1)

    return df_mrgd


In [17]:
# path to the output of opus to csv tool AB_quantized_spectra.csv 
# path_to_ab_spectra = Path('../DS-ML69 product1 spectra/20221222-095217')
df_spectra_1 = pd.read_csv("D://CropNutsDocuments/MSSC_DVC/data/spc/spc.csv", engine='c')
# path_to_ab_spectra = Path('../DS_L19/20230320-091519')

# df_spectra_1 = pd.read_csv(path_to_ab_spectra / 'AB_quantized_spectra.csv',index_col=0)

In [5]:
# # path to the output of opus to csv tool AB_quantized_spectra.csv 
# path_to_ab_spectra = Path('../DS-ML88')

# # path_to_ab_spectra = Path('C:/Users/Tsuma Thomas/Documents/CropNutsDocuments/DS-GIS46/DS-GIS46 all-batches-opus-pre-sorted_22-12-21')

# df_spectra_1 = pd.read_csv(path_to_ab_spectra / '19_8_2022_global_spectra_datav2dot2_with_rw.csv',index_col=0)

In [6]:
df_spectra_1

,Unnamed: 0,522,524,526,528,530,532,534,536,538,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
0,AAGD207-408SA0658,2.313672,2.433634,2.516725,2.498609,2.422177,2.321191,2.234420,2.191601,2.213712,...,1.475988,1.473906,1.472432,1.471783,1.471769,1.471628,1.471268,1.470556,1.469374,1.467965
1,AAGD207-408SA0659,2.395613,2.345511,2.301150,2.292676,2.291118,2.284511,2.263340,2.240161,2.230329,...,1.460731,1.460398,1.460315,1.460095,1.459363,1.458242,1.457019,1.455912,1.455504,1.456098
2,AAGD207-408SA0660,2.371531,2.393945,2.338019,2.289904,2.304425,2.286844,2.216206,2.181717,2.189945,...,1.317696,1.317353,1.316663,1.316057,1.315726,1.315082,1.314158,1.313547,1.313124,1.312351
3,AAGD207-408SA0661,2.261925,2.232464,2.204991,2.198462,2.215577,2.224306,2.207614,2.179986,2.161271,...,1.453321,1.452374,1.451718,1.450873,1.449287,1.447532,1.446346,1.445924,1.446245,1.446643
4,AAGD207-408SA0662,2.350329,2.312178,2.301096,2.309662,2.300066,2.288580,2.295082,2.307477,2.280081,...,1.648263,1.647880,1.649471,1.650738,1.651190,1.651116,1.649356,1.647050,1.645815,1.644683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89493,CW104SA0052,1.915089,1.907579,2.006204,2.047937,2.043367,2.039258,2.045996,2.070376,2.097082,...,1.735016,1.731109,1.727900,1.728896,1.729304,1.726673,1.724738,1.723750,1.723889,1.726787
89494,CW104SA0053,2.077128,2.040786,2.043095,2.071373,2.064174,2.076678,2.131703,2.123007,2.021653,...,1.437869,1.437094,1.434376,1.433937,1.436358,1.438110,1.439149,1.439427,1.436281,1.432167
89495,FA218-24SA0001,1.954683,1.999012,2.016380,2.021764,1.998302,1.951551,1.906951,1.885720,1.889962,...,1.155097,1.155336,1.155303,1.155395,1.155574,1.155414,1.155130,1.155152,1.155191,1.155193
89496,FA218-24SA0002,2.142205,2.156203,2.134077,2.083162,2.059813,2.066087,2.080873,2.077663,2.037589,...,1.089436,1.089234,1.089074,1.089026,1.088915,1.088838,1.088641,1.088033,1.087544,1.087490


In [7]:
df_spectra_1

,Unnamed: 0,522,524,526,528,530,532,534,536,538,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
0,AAGD207-408SA0658,2.313672,2.433634,2.516725,2.498609,2.422177,2.321191,2.234420,2.191601,2.213712,...,1.475988,1.473906,1.472432,1.471783,1.471769,1.471628,1.471268,1.470556,1.469374,1.467965
1,AAGD207-408SA0659,2.395613,2.345511,2.301150,2.292676,2.291118,2.284511,2.263340,2.240161,2.230329,...,1.460731,1.460398,1.460315,1.460095,1.459363,1.458242,1.457019,1.455912,1.455504,1.456098
2,AAGD207-408SA0660,2.371531,2.393945,2.338019,2.289904,2.304425,2.286844,2.216206,2.181717,2.189945,...,1.317696,1.317353,1.316663,1.316057,1.315726,1.315082,1.314158,1.313547,1.313124,1.312351
3,AAGD207-408SA0661,2.261925,2.232464,2.204991,2.198462,2.215577,2.224306,2.207614,2.179986,2.161271,...,1.453321,1.452374,1.451718,1.450873,1.449287,1.447532,1.446346,1.445924,1.446245,1.446643
4,AAGD207-408SA0662,2.350329,2.312178,2.301096,2.309662,2.300066,2.288580,2.295082,2.307477,2.280081,...,1.648263,1.647880,1.649471,1.650738,1.651190,1.651116,1.649356,1.647050,1.645815,1.644683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89493,CW104SA0052,1.915089,1.907579,2.006204,2.047937,2.043367,2.039258,2.045996,2.070376,2.097082,...,1.735016,1.731109,1.727900,1.728896,1.729304,1.726673,1.724738,1.723750,1.723889,1.726787
89494,CW104SA0053,2.077128,2.040786,2.043095,2.071373,2.064174,2.076678,2.131703,2.123007,2.021653,...,1.437869,1.437094,1.434376,1.433937,1.436358,1.438110,1.439149,1.439427,1.436281,1.432167
89495,FA218-24SA0001,1.954683,1.999012,2.016380,2.021764,1.998302,1.951551,1.906951,1.885720,1.889962,...,1.155097,1.155336,1.155303,1.155395,1.155574,1.155414,1.155130,1.155152,1.155191,1.155193
89496,FA218-24SA0002,2.142205,2.156203,2.134077,2.083162,2.059813,2.066087,2.080873,2.077663,2.037589,...,1.089436,1.089234,1.089074,1.089026,1.088915,1.088838,1.088641,1.088033,1.087544,1.087490


In [8]:
len(df_spectra_1)

89498

In [9]:
# extract sample code
df_spectra_1.index = df_spectra_1.index.str.extract('([C]\w{3,5}SA\w{2,5}|[F]\w{4,5}[-]{1}\w{6,9}|A\w{5,7}[-]{1}\w{7,10}|\w{3}[-]{1}\w{1,2}[-]{1}\w{7,10}|\w{3}[-]{1}\w{2}[-]{1}\w{4,6})', expand=False)

AttributeError: Can only use .str accessor with string values!

In [ ]:
df_spectra_1 = df_spectra_1.reset_index()
df_spectra_1.dropna(axis=0, inplace=True)
df_spectra_1 = df_spectra_1.set_index("sample_id")

In [18]:
# df_spectra_1 = df_spectra_1.rename(columns={"Unnamed: 0":"sample_id"})
# df_spectra_1 = df_spectra_1.set_index("sample_id")

In [ ]:
# df_spectra_1.index = df_spectra_1.index.astype("str")

In [19]:
df_spectra_1

,522,524,526,528,530,532,534,536,538,540,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
sample_id,,,,,,,,,,,,,,,,,,,,,
AGD207-408SA0706,2.096135,2.074343,2.077232,2.100555,2.109029,2.113436,2.103539,2.076615,2.058088,2.052834,...,1.397774,1.396845,1.396773,1.396401,1.395748,1.395857,1.395875,1.394762,1.394079,1.393831
AGD207-431SA0001,2.154667,2.128691,2.090363,2.061056,2.036317,2.019021,2.018069,2.030793,2.041479,2.035714,...,1.319761,1.319318,1.319803,1.319366,1.317630,1.316362,1.315863,1.315292,1.314809,1.314864
AGD207-431SA0002,2.412217,2.413204,2.360303,2.307860,2.296765,2.316080,2.321781,2.294391,2.270676,2.284707,...,1.306064,1.306364,1.306144,1.305445,1.305173,1.305272,1.304674,1.303443,1.302389,1.301506
AGD207-431SA0003,2.059137,2.093001,2.136247,2.157497,2.152817,2.144714,2.131581,2.108572,2.105603,2.127556,...,1.306060,1.305862,1.305825,1.305172,1.304238,1.303409,1.302201,1.300902,1.300176,1.300167
AGD207-431SA0004,2.279650,2.237250,2.218038,2.231755,2.238702,2.222442,2.208411,2.211492,2.218197,2.235896,...,1.184210,1.183787,1.183616,1.183282,1.182663,1.182083,1.181676,1.181184,1.180578,1.180207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CS301SA0020,2.045630,2.069787,2.128691,2.186521,2.180089,2.127234,2.084637,2.062834,2.060205,2.083387,...,1.202821,1.202508,1.201742,1.199615,1.198251,1.199463,1.201781,1.202746,1.201712,1.200562
CS301SA0021,2.130854,2.172474,2.172629,2.144991,2.153000,2.181448,2.145078,2.083674,2.087624,2.123100,...,1.345972,1.344010,1.341521,1.339447,1.338350,1.338700,1.340493,1.342617,1.343494,1.342798
CS301SA0023,2.174167,2.143292,2.162553,2.200258,2.199128,2.174309,2.169863,2.175358,2.139927,2.106794,...,1.248387,1.249193,1.250899,1.250782,1.248707,1.246431,1.244417,1.243429,1.244811,1.248418


In [20]:
# df_avg_1 = mean_dupes_drop_singles_filter_by_std(df_spectra_1)
# df = mean_dupes_drop_singles_filter_by_std(df_spectra_1)
df = df_spectra_1

In [21]:
df

,522,524,526,528,530,532,534,536,538,540,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
sample_id,,,,,,,,,,,,,,,,,,,,,
AGD207-408SA0706,2.096135,2.074343,2.077232,2.100555,2.109029,2.113436,2.103539,2.076615,2.058088,2.052834,...,1.397774,1.396845,1.396773,1.396401,1.395748,1.395857,1.395875,1.394762,1.394079,1.393831
AGD207-431SA0001,2.154667,2.128691,2.090363,2.061056,2.036317,2.019021,2.018069,2.030793,2.041479,2.035714,...,1.319761,1.319318,1.319803,1.319366,1.317630,1.316362,1.315863,1.315292,1.314809,1.314864
AGD207-431SA0002,2.412217,2.413204,2.360303,2.307860,2.296765,2.316080,2.321781,2.294391,2.270676,2.284707,...,1.306064,1.306364,1.306144,1.305445,1.305173,1.305272,1.304674,1.303443,1.302389,1.301506
AGD207-431SA0003,2.059137,2.093001,2.136247,2.157497,2.152817,2.144714,2.131581,2.108572,2.105603,2.127556,...,1.306060,1.305862,1.305825,1.305172,1.304238,1.303409,1.302201,1.300902,1.300176,1.300167
AGD207-431SA0004,2.279650,2.237250,2.218038,2.231755,2.238702,2.222442,2.208411,2.211492,2.218197,2.235896,...,1.184210,1.183787,1.183616,1.183282,1.182663,1.182083,1.181676,1.181184,1.180578,1.180207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CS301SA0020,2.045630,2.069787,2.128691,2.186521,2.180089,2.127234,2.084637,2.062834,2.060205,2.083387,...,1.202821,1.202508,1.201742,1.199615,1.198251,1.199463,1.201781,1.202746,1.201712,1.200562
CS301SA0021,2.130854,2.172474,2.172629,2.144991,2.153000,2.181448,2.145078,2.083674,2.087624,2.123100,...,1.345972,1.344010,1.341521,1.339447,1.338350,1.338700,1.340493,1.342617,1.343494,1.342798
CS301SA0023,2.174167,2.143292,2.162553,2.200258,2.199128,2.174309,2.169863,2.175358,2.139927,2.106794,...,1.248387,1.249193,1.250899,1.250782,1.248707,1.246431,1.244417,1.243429,1.244811,1.248418


In [14]:
df_spectra_1.index

Index(['AAGD207-408SA0658', 'AAGD207-408SA0659', 'AAGD207-408SA0660',
       'AAGD207-408SA0661', 'AAGD207-408SA0662', 'AAGD207-408SA0663',
       'AAGD207-408SA0664', 'AAGD207-408SA0665', 'AAGD207-408SA0666',
       'AAGD207-408SA0667',
       ...
       'CW017SA7161', 'CW017SA7162', 'CW017SA7164', 'CW017SA7167',
       'CW017SA7168', 'CW104SA0052', 'CW104SA0053', 'FA218-24SA0001',
       'FA218-24SA0002', 'FA218-24SA0003'],
      dtype='object', name='sample_id', length=89498)

In [15]:
# df.to_csv("C:/Users/Tsuma Thomas/Documents/CropNutsDocuments/DS-ML69 product1 spectra/20221206-103534.csv")

In [22]:
# save to path of notebook
df.to_csv('spc.csv')

In [23]:
# path to file
filename = 'spc.csv'

In [33]:
chemicals = ['aluminium', 
            'phosphorus', 'ph', 'exchangeable_acidity', 'calcium', 'magnesium',
              'sulphur', 'sodium', 'iron', 'manganese', 'boron', 'copper', 'zinc', 'total_nitrogen', 'potassium',
             'ec_salts', 'organic_carbon', 'cec', 'sand', 'silt', 'clay']

# chemicals = ['exchangeable_acidity','clay']


In [25]:
from math import log10, floor
def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1)

In [26]:
# read data from file for predictions
data = pd.read_csv(filename, index_col=0, engine='c')

In [27]:
data = data.set_index("sample_id")

KeyError: "None of ['sample_id'] are in the columns"

In [28]:
data

,522,524,526,528,530,532,534,536,538,540,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
sample_id,,,,,,,,,,,,,,,,,,,,,
AGD207-408SA0706,2.096135,2.074343,2.077232,2.100555,2.109029,2.113436,2.103539,2.076615,2.058088,2.052834,...,1.397774,1.396845,1.396773,1.396401,1.395748,1.395857,1.395875,1.394762,1.394079,1.393831
AGD207-431SA0001,2.154667,2.128691,2.090363,2.061056,2.036317,2.019021,2.018069,2.030793,2.041479,2.035714,...,1.319761,1.319318,1.319803,1.319366,1.317630,1.316362,1.315863,1.315292,1.314809,1.314864
AGD207-431SA0002,2.412217,2.413204,2.360303,2.307860,2.296765,2.316080,2.321781,2.294391,2.270676,2.284707,...,1.306064,1.306364,1.306144,1.305445,1.305173,1.305272,1.304674,1.303443,1.302389,1.301506
AGD207-431SA0003,2.059137,2.093001,2.136247,2.157497,2.152817,2.144714,2.131581,2.108572,2.105603,2.127556,...,1.306060,1.305862,1.305825,1.305172,1.304238,1.303409,1.302201,1.300902,1.300176,1.300167
AGD207-431SA0004,2.279650,2.237250,2.218038,2.231755,2.238702,2.222442,2.208411,2.211492,2.218197,2.235896,...,1.184210,1.183787,1.183616,1.183282,1.182663,1.182083,1.181676,1.181184,1.180578,1.180207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CS301SA0020,2.045630,2.069787,2.128691,2.186521,2.180089,2.127234,2.084637,2.062834,2.060205,2.083387,...,1.202821,1.202508,1.201742,1.199615,1.198251,1.199463,1.201781,1.202746,1.201712,1.200562
CS301SA0021,2.130854,2.172474,2.172629,2.144991,2.153000,2.181448,2.145078,2.083674,2.087624,2.123100,...,1.345972,1.344010,1.341521,1.339447,1.338350,1.338700,1.340493,1.342617,1.343494,1.342798
CS301SA0023,2.174167,2.143292,2.162553,2.200258,2.199128,2.174309,2.169863,2.175358,2.139927,2.106794,...,1.248387,1.249193,1.250899,1.250782,1.248707,1.246431,1.244417,1.243429,1.244811,1.248418


In [29]:
# make sure we have 1728 wave numbers
data = data.T.head(1728).T

In [30]:
data.index

Index(['AGD207-408SA0706', 'AGD207-431SA0001', 'AGD207-431SA0002',
       'AGD207-431SA0003', 'AGD207-431SA0004', 'AGD207-431SA0005',
       'AGD207-433SA0001', 'AGD207-433SA0002', 'AGD207-433SA0003',
       'AGD207-433SA0004',
       ...
       'CS301SA0012', 'CS301SA0016', 'CS301SA0017', 'CS301SA0018',
       'CS301SA0019', 'CS301SA0020', 'CS301SA0021', 'CS301SA0023',
       'CW104SA0052', 'CW104SA0053'],
      dtype='object', name='sample_id', length=82126)

In [31]:
# path to models and preprocessing steps
# base_path = Path('./dl_models_all_chems_20210414/saved_models')
base_path = Path('./dl_models_all_chems_20210414/dl_v2.2_update_2022')

In [34]:
for chemical in chemicals:
    print(chemical)
    preds_comb = pd.DataFrame()
    models_folder = base_path / chemical / 'std'
    all_models = [x for x in models_folder.glob('**/*.hdf5')]

    
    new_indices = data.index



    for model_path in all_models:

        json_path = model_path.parent.parent / 'model.json'

        with open(json_path) as f:
            json_ = json.load(f)

        inputs = []

        for i in range(len(json_['Inputs'])):
            input_name = json_['Inputs'][i]['Name']
            train = data.copy(deep=True)

            for j in range(len(json_['Inputs'][i]['Pre-processing'])):
                key_ = json_['Inputs'][i]['Pre-processing'][j]['Name']
                if input_name == 'nir2':
                    input_name = 'nir.2'
                pickle_path = model_path.parent / 'preprocess' / f'input.{input_name}.{j}.{key_}.pickle'
                pickle_ = joblib.load(pickle_path)
                train = pickle_.fit_transform(train)

            inputs.append(train.values)

        tf.keras.backend.clear_session()
        model = tf.keras.models.load_model(model_path)
        preds = pd.DataFrame(model(inputs).numpy())
        preds_comb = pd.concat([preds_comb, preds], axis=1)

    preds_comb = preds_comb.median(axis=1)
    preds_comb.index = new_indices
    
    
    # save output
    preds_comb.to_csv(f'D://CropNutsDocuments/DS-ML87/outputFiles/preds/{chemical}.csv')

aluminium
phosphorus
ph
exchangeable_acidity
calcium
magnesium
sulphur
sodium
iron
manganese
boron
copper
zinc
total_nitrogen
potassium
ec_salts
organic_carbon
cec
sand
silt
clay


In [ ]:
print(json_['Inputs'][i])